# Pockitect Colab Training (Unsloth)

This notebook fine-tunes the model on a GPU-backed Colab runtime using the project training data.

**Before you start:** Runtime → Change runtime type → **GPU**

## Step 1: Clone the Repository

First, we clone the pockitect-mvp repository into the Colab environment. This brings in all the project code, training data, and scripts needed for fine-tuning.

In [ ]:
%cd /content
!rm -rf pockitect-mvp
!git clone https://github.com/BrendenKennedy/pockitect-mvp.git
%cd /content/pockitect-mvp

## Step 2: Install Dependencies

Install Unsloth (a fast fine-tuning library) and all project dependencies. Unsloth is optimized for Colab and provides significant speed improvements for training.

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -r requirements.txt

## Step 3: Validate Training Data

Before training, we validate the training data to ensure it's properly formatted and contains no errors. This helps catch issues early before starting the expensive fine-tuning process.

In [ ]:
!python tools/validate_training_data.py

## Step 4: Convert to Fine-tuning Format

Convert the training data into Unsloth's preferred JSONL format. This format is optimized for efficient loading during training and ensures the data is structured correctly for the fine-tuning process.

In [ ]:
!python tools/convert_to_finetuning_format.py --format unsloth

## Step 5: Fine-tune the Model

This is the main training step. We fine-tune the Qwen2.5-3B model using the converted training data. The configuration is optimized for T4 GPUs (15GB VRAM) with:

- **Batch size**: 1 (to fit in memory)
- **Gradient accumulation**: 8 (effective batch size of 8)
- **Max sequence length**: 1024 tokens
- **Early stopping**: Stops training if validation loss doesn't improve for 3 evaluations
- **Evaluation steps**: Validates every 25 training steps

This step may take 30-60 minutes depending on your GPU.

In [ ]:
# Memory-optimized for T4 (15GB VRAM)
!python tools/finetune_model.py \
    --model unsloth/Qwen2.5-3B \
    --train data/finetuning/train_unsloth.jsonl \
    --epochs 3 \
    --batch-size 1 \
    --gradient-accumulation 8 \
    --max-seq-len 1024 \
    --early-stopping \
    --early-stopping-patience 3 \
    --eval-steps 25

## Step 6: Export to GGUF Format

After training completes, we export the fine-tuned model to GGUF format (Q4_K_M quantization). This format:

- Reduces model size significantly (from ~6GB to ~2GB)
- Maintains good quality with minimal performance loss
- Is compatible with Ollama and other inference engines
- Can run efficiently on consumer hardware

The exported GGUF file will be saved in `data/finetuning/output/gguf/`.

In [ ]:
import os

# First check if the adapter exists
adapter_path = "data/finetuning/output/adapter"
if not os.path.exists(adapter_path):
    print(f"❌ Adapter not found at {adapter_path}")
    print("Make sure training completed successfully (step 5)")
else:
    print(f"✅ Found adapter at {adapter_path}")
    print("Files:", os.listdir(adapter_path))

    from unsloth import FastLanguageModel

    # Load the fine-tuned model
    print("\n📥 Loading fine-tuned model...")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=adapter_path,
        max_seq_length=1024,
        dtype=None,
        load_in_4bit=True,
    )

    # Save as GGUF (Q4_K_M is a good balance of size and quality)
    print("\n📦 Exporting to GGUF (this may take a few minutes)...")
    output_dir = "data/finetuning/output/gguf"
    model.save_pretrained_gguf(
        output_dir,
        tokenizer,
        quantization_method="q4_k_m"
    )

    # List what was created
    print("\n📂 Created files:")
    if os.path.exists(output_dir):
        for f in os.listdir(output_dir):
            size_mb = os.path.getsize(os.path.join(output_dir, f)) / (1024*1024)
            print(f"  - {f} ({size_mb:.1f} MB)")
    print("\n✅ GGUF export complete!")

In [ ]:
## 7) Download the GGUF model

In [ ]:
!find /content -name "*.gguf" -type f 2>/dev/null

## 8) Create Ollama model locally

After downloading the GGUF file, run on your local machine:

```bash
# Create a Modelfile
echo 'FROM ./pockitect-unsloth.Q4_K_M.gguf' > Modelfile
echo 'PARAMETER temperature 0.2' >> Modelfile

# Create the Ollama model
ollama create pockitect:finetuned -f Modelfile

# Test it
ollama run pockitect:finetuned "I need a simple blog server"
```